In [32]:
pip install -q transformers

In [33]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [34]:
context1 = r"""🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
    architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
    Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
    TensorFlow 2.0 and PyTorch."""

contexts = [

r"""The four largest cities in the Netherlands are Amsterdam, Rotterdam, The Hague and Utrecht.[17] Amsterdam is the country's most populous city 
 and nominal capital,[18] while The Hague holds the seat of the States General, Cabinet and Supreme Court.[19] The Port of Rotterdam is the busiest 
 seaport in Europe, and the busiest in any country outside East Asia and Southeast Asia, behind only China and Singapore.""",
r"""Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
 question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
 a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.""",
"The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.",
r"""MOSCOW, Russia (CNN) – Russian space officials say the crew of the Soyuz space ship is resting after a rough ride back to Earth."
  A South Korean bioengineer was one of three people on board the Soyuz capsule. The craft carrying South Korea’s first astronaut 
  landed in northern Kazakhstan on Saturday, 260 miles (418 kilometers) off its mark, they said. Mission Control spokesman Valery 
  Lyndin said the condition of the crew – South Korean bioengineer Yi So-yeon, American astronaut Peggy Whitson and Russian flight 
  engineer Yuri Malenchenko – was satisfactory, though the three had been subjected to severe G-forces during the re-entry.""",
context1, context1
]

questions = [
"What is the capital of the Netherlands?",
"What is a good example of a question answering dataset?",
"Why is model conversion important?",
"Where did the Soyuz capsule land?",
"How many pretrained models are available in 🤗 Transformers?",
"🤗 Transformers provides interoperability between which frameworks?"
]

In [35]:
model_names = ["deepset/roberta-base-squad2", "bert-large-uncased-whole-word-masking-finetuned-squad", \
               "ahotrod/electra_large_discriminator_squad2_512"]

In [36]:
# Suppress irrelevant warning

# To control logging level for various modules used in the application:
import logging
import re
def set_global_logging_level(level=logging.ERROR, prefices=[""]):
    """
    Override logging levels of different modules based on their name as a prefix.
    It needs to be invoked after the modules have been loaded so that their loggers have been initialized.

    Args:
        - level: desired level. e.g. logging.INFO. Optional. Default is logging.ERROR
        - prefices: list of one or more str prefices to match (e.g. ["transformers", "torch"]). Optional.
          Default is `[""]` to match all active loggers.
          The match is a case-sensitive `module_name.startswith(prefix)`
    """
    prefix_re = re.compile(fr'^(?:{ "|".join(prefices) })')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)

set_global_logging_level(logging.ERROR)

In [37]:
print(f"Predicting Answers to {len(questions)} Questions using {len(model_names)} different models...\n")

predicted_answers = [[0 for x in range(len(model_names))] for y in range(len(questions))] 
predicted_scores = [[0 for x in range(len(model_names))] for y in range(len(questions))] 

for model_index in range(len(model_names)):
    # Retrieve the pre-trained model. Slow.
    nlp = pipeline('question-answering', model=model_names[model_index], tokenizer=model_names[model_index])

    question_index = 0
    for question, context in zip(questions, contexts):
        # Predict the answer
        answer = nlp(question=question, context=context)
        predicted_answers[question_index][model_index] = answer['answer']
        predicted_scores[question_index][model_index] = answer['score']
        question_index += 1

for question_index in range(0, len(questions)):
    print(f'Question: {questions[question_index]}')
    for model_index in range(0, len(model_names)):
        print(f"Model: {model_names[model_index]} Answer: '{predicted_answers[question_index][model_index]}'")
    print("\n")

Predicting Answers to 6 Questions using 3 different models...

Question: What is the capital of the Netherlands?
Model: deepset/roberta-base-squad2 Answer: 'Amsterdam'
Model: bert-large-uncased-whole-word-masking-finetuned-squad Answer: 'Amsterdam'
Model: ahotrod/electra_large_discriminator_squad2_512 Answer: 'Amsterdam'


Question: What is a good example of a question answering dataset?
Model: deepset/roberta-base-squad2 Answer: 'SQuAD dataset'
Model: bert-large-uncased-whole-word-masking-finetuned-squad Answer: 'SQuAD dataset'
Model: ahotrod/electra_large_discriminator_squad2_512 Answer: 'SQuAD dataset'


Question: Why is model conversion important?
Model: deepset/roberta-base-squad2 Answer: 'gives freedom to the user'
Model: bert-large-uncased-whole-word-masking-finetuned-squad Answer: 'gives freedom to the user'
Model: ahotrod/electra_large_discriminator_squad2_512 Answer: 'gives freedom to the user and let people easily switch between frameworks'


Question: Where did the Soyuz ca

Slightly lower level Huggingface implementation

Some good explanations can be found here: https://towardsdatascience.com/question-and-answering-with-bert-6ef89a78dac

In [38]:
print("Predicting answer using Electra model pretrained on squad2\n")

import torch
model_name = 'ahotrod/electra_large_discriminator_squad2_512'
# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs = tokenizer(questions[0], contexts[0], add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0] # the list of all indices of words in question + context
    
text_tokens = tokenizer.convert_ids_to_tokens(input_ids) # Get the tokens for the question + context
answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)

answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    
print(f"Question: {questions[0]}")
print(f"Answer: {answer}")

Predicting answer using Electra model pretrained on squad2

Question: What is the capital of the Netherlands?
Answer: amsterdam
